***GENERATED CODE FOR 3110sqlsink3 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 3110sqlsink3Hooks.ipynb
try:
	#sourcePreExecutionHook()

	bankdepositanalysistl = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Bank_Deposit_Analysis_TL.csv', 'filename': 'Bank_Deposit_Analysis_TL.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PERFORMING OPERATIONS***

In [ ]:
#%run 3110sqlsink3Hooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bankdepositanalysistl,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/Bank_Deposit_Analysis_TL.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/Bank_Deposit_Analysis_TL.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/6540db8f8f9ca5b9286736891698749571/0part.csv", "requestRatio": 0.0, "totalRows": 24, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 18, "numberOfRows": 24, "duplicateRowCount": 0, "stats": [{"column": "age", "alias": "age", "generated": 0, "type": "numeric", "max": 78, "min": 25, "mean": 42.708333333333336, "missing": 0.0, "stddev": 13.64, "outliers": [78], "validation": []}, {"column": "job", "alias": "job", "generated": 0, "type": "String", "max": "unemployed", "min": "admin.", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "marital", "alias": "marital", "generated": 0, "type": "String", "max": "single", "min": "divorced", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "education", "alias": "education", "generated": 0, "type": "String", "max": "unknown", "min": "primary", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "default", "alias": "default", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "balance", "alias": "balance", "generated": 0, "type": "numeric", "max": 7944, "min": -199, "mean": 1405.875, "missing": 0.0, "stddev": 2233.54, "outliers": [5943, 4389, 7944, 3696, 5091], "validation": []}, {"column": "housing", "alias": "housing", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "loan", "alias": "loan", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "contact", "alias": "contact", "generated": 0, "type": "String", "max": "unknown", "min": "cellular", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "day", "alias": "day", "generated": 0, "type": "numeric", "max": 29, "min": 1, "mean": 17.791666666666668, "missing": 0.0, "stddev": 8.72, "outliers": [], "validation": []}, {"column": "month", "alias": "month", "generated": 0, "type": "String", "max": "oct", "min": "apr", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "duration", "alias": "duration", "generated": 0, "type": "numeric", "max": 618, "min": 89, "mean": 232.625, "missing": 0.0, "stddev": 124.99, "outliers": [618], "validation": []}, {"column": "campaign", "alias": "campaign", "generated": 0, "type": "numeric", "max": 5, "min": 1, "mean": 1.6666666666666667, "missing": 0.0, "stddev": 1.05, "outliers": [4, 5], "validation": []}, {"column": "pdays", "alias": "pdays", "generated": 0, "type": "numeric", "max": 299, "min": -1, "mean": 40.916666666666664, "missing": 0.0, "stddev": 90.54, "outliers": [181, 180, 81, 260, 299], "validation": []}, {"column": "previous", "alias": "previous", "generated": 0, "type": "numeric", "max": 10, "min": 0, "mean": 0.7916666666666666, "missing": 0.0, "stddev": 2.17, "outliers": [3, 3, 1, 2, 10], "validation": []}, {"column": "poutcome", "alias": "poutcome", "generated": 0, "type": "String", "max": "unknown", "min": "failure", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "deposit", "alias": "deposit", "generated": 0, "type": "String", "max": "yes", "min": "no", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Campaign_P", "alias": "Campaign_P", "generated": 1, "type": "numeric", "max": 15, "min": 1, "mean": 2.4583333333333335, "missing": 0.0, "stddev": 2.87, "outliers": [15], "validation": []}]}, "predictionPowerScore": [{"Campaign_P": 1.0, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.1751803752}, {"Campaign_P": 0.0, "age": 1.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.082345191, "poutcome": 0.0, "previous": 0.0487483531}, {"Campaign_P": 0.0, "age": 0.0, "balance": 1.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.40625, "age": 0.0, "balance": 0.0, "campaign": 1.0, "contact": 0.1488095238, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0048365428, "age": 0.0, "balance": 0.0462506842, "campaign": 0.1776334776, "contact": 1.0, "day": 0.0916940339, "default": 0.0048365428, "deposit": 0.0048365428, "duration": 0.0, "education": 0.0048365428, "housing": 0.0048365428, "job": 0.0, "loan": 0.0048365428, "marital": 0.0048365428, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0048365428}, {"Campaign_P": 0.0, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 1.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0233766234, "age": 0.0, "balance": 0.0, "campaign": 0.0233766234, "contact": 0.0233766234, "day": 0.0, "default": 1.0, "deposit": 0.0233766234, "duration": 0.0233766234, "education": 0.0233766234, "housing": 0.0233766234, "job": 0.0233766234, "loan": 0.0233766234, "marital": 0.0233766234, "month": 0.0233766234, "pdays": 0.0233766234, "poutcome": 0.0233766234, "previous": 0.0233766234}, {"Campaign_P": 0.1714285714, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.1410714286, "day": 0.0, "default": 0.0, "deposit": 1.0, "duration": 0.0, "education": 0.4142857143, "housing": 0.393452381, "job": 0.0, "loan": 0.0, "marital": 0.3074074074, "month": 0.5619047619, "pdays": 0.0583333333, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 1.0, "education": 0.0, "housing": 0.077412761, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.443697479, "age": 0.0, "balance": 0.1673669468, "campaign": 0.0763305322, "contact": 0.0231092437, "day": 0.0343137255, "default": 0.0, "deposit": 0.3271708683, "duration": 0.0, "education": 1.0, "housing": 0.3333333333, "job": 0.4148459384, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.1645658263, "poutcome": 0.0, "previous": 0.2107843137}, {"Campaign_P": 0.0, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 1.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.1481481481, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0560606061, "age": 0.0, "balance": 0.0, "campaign": 0.0378787879, "contact": 0.0, "day": 0.053030303, "default": 0.0, "deposit": 0.0, "duration": 0.0606060606, "education": 0.1272727273, "housing": 0.0363636364, "job": 1.0, "loan": 0.0584415584, "marital": 0.0, "month": 0.0909090909, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0, "age": 0.6363636364, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 1.0, "marital": 0.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0, "age": 0.2666666667, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 1.0, "month": 0.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.0, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.02, "default": 0.0, "deposit": 0.0783333333, "duration": 0.0883333333, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.2416666667, "month": 1.0, "pdays": 0.0, "poutcome": 0.0, "previous": 0.0}, {"Campaign_P": 0.2440357853, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 1.0, "poutcome": 0.7246520875, "previous": 0.6207753479}, {"Campaign_P": 0.2532795699, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0063343109, "deposit": 0.0063343109, "duration": 0.0, "education": 0.0, "housing": 0.0063343109, "job": 0.0, "loan": 0.0430433366, "marital": 0.0063343109, "month": 0.0, "pdays": 0.7996415771, "poutcome": 1.0, "previous": 0.5222222222}, {"Campaign_P": 0.2368421053, "age": 0.0, "balance": 0.0, "campaign": 0.0, "contact": 0.0, "day": 0.0, "default": 0.0, "deposit": 0.0, "duration": 0.0, "education": 0.0, "housing": 0.0, "job": 0.0, "loan": 0.0, "marital": 0.0, "month": 0.0, "pdays": 0.1052631579, "poutcome": 0.0, "previous": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "campaign", "type": "numeric", "min": "1.0", "max": "5.0", "mean": "1.7"}, {"columnName": "previous", "type": "numeric", "min": "0.0", "max": "10.0", "mean": "0.8"}], "functionsData": [{"operand1": "campaign", "operator": "add", "operand2": "previous", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "campaign", "type": "numeric", "min": "1.0", "max": "5.0", "mean": "1.7"}}, {"Columndata": {"columnName": "previous", "type": "numeric", "min": "0.0", "max": "10.0", "mean": "0.8"}}, {"Columndata": ""}], "newColumnName": "Campaign_P"}]}], "functionChanges": [{"columnName": "campaign", "functionName": "Formula", "Type": "numeric", "Parameters": [{"operand1": "campaign", "operator": "add", "operand2": "previous", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "campaign", "type": "numeric", "min": "1.0", "max": "5.0", "mean": "1.7"}}, {"Columndata": {"columnName": "previous", "type": "numeric", "min": "0.0", "max": "10.0", "mean": "0.8"}}, {"Columndata": ""}], "newColumnName": "Campaign_P"}]}, {"columnName": "previous", "functionName": "Formula", "Type": "numeric", "Parameters": [{"operand1": "campaign", "operator": "add", "operand2": "previous", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "campaign", "type": "numeric", "min": "1.0", "max": "5.0", "mean": "1.7"}}, {"Columndata": {"columnName": "previous", "type": "numeric", "min": "0.0", "max": "10.0", "mean": "0.8"}}, {"Columndata": ""}], "newColumnName": "Campaign_P"}]}], "fileheader": [{"field": "age", "alias": "age", "generated": 0, "position": 1, "type": "numeric"}, {"field": "job", "alias": "job", "generated": 0, "position": 2, "type": "String"}, {"field": "marital", "alias": "marital", "generated": 0, "position": 3, "type": "String"}, {"field": "education", "alias": "education", "generated": 0, "position": 4, "type": "String"}, {"field": "default", "alias": "default", "generated": 0, "position": 5, "type": "String"}, {"field": "balance", "alias": "balance", "generated": 0, "position": 6, "type": "numeric"}, {"field": "housing", "alias": "housing", "generated": 0, "position": 7, "type": "String"}, {"field": "loan", "alias": "loan", "generated": 0, "position": 8, "type": "String"}, {"field": "contact", "alias": "contact", "generated": 0, "position": 9, "type": "String"}, {"field": "day", "alias": "day", "generated": 0, "position": 10, "type": "numeric"}, {"field": "month", "alias": "month", "generated": 0, "position": 11, "type": "String"}, {"field": "duration", "alias": "duration", "generated": 0, "position": 12, "type": "numeric"}, {"field": "campaign", "alias": "campaign", "generated": 0, "position": 13, "type": "numeric"}, {"field": "pdays", "alias": "pdays", "generated": 0, "position": 14, "type": "numeric"}, {"field": "previous", "alias": "previous", "generated": 0, "position": 15, "type": "numeric"}, {"field": "poutcome", "alias": "poutcome", "generated": 0, "position": 16, "type": "String"}, {"field": "deposit", "alias": "deposit", "generated": 0, "position": 17, "type": "String"}, {"field": "Campaign_P", "alias": "Campaign_P", "generated": 1, "position": 18, "type": "numeric"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run 3110sqlsink3Hooks.ipynb
try:
	#sinkPreExecutionHook()

	sqlserver = RDBMSConnector.put(spark, "{'host': '3.17.87.4', 'port': '1433', 'password': 'Passw0rd@123', 'database': 'testdb', 'user': 'SA', 'optionalDB': '', 'table_option': 'create_new_table', 'table': 'BankDSQL', 'new_table_name': 'BankDSQL', 'writemode': '', 'is_header': 'Use Header Line', 'use_generatedColumns': False, 'generatedColumns': [], 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sinkPostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)
